<a href="https://colab.research.google.com/github/greyes69/Sentiment-Analysis-Project/blob/main/SentimentMovieReviewsSVCVersion1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import string
import joblib
import re
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Access files in your Google Drive
df = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

# Assuming you have your dataset in a pandas DataFrame with two columns: 'reviews' and 'label'
#df = pd.read_csv('IMDB Dataset.csv')  # Uncomment and load your dataset accordingly


In [11]:
df = df.rename(columns={'review': 'reviews'})

In [12]:
# Custom list of very frequent words (you can update this list based on your dataset)
custom_stop_words = set(['movie', 'film', 'character', 'plot'])  # Add domain-specific frequent words if needed
all_stop_words = list(ENGLISH_STOP_WORDS.union(custom_stop_words))

In [13]:
# Function to preprocess text: remove symbols, stop words, frequent words, etc.
def preprocess_text(text):
    # 1. Convert text to lowercase
    text = text.lower()

    # 2. Remove punctuation and special characters using regex
    text = re.sub(r'[^\w\s]', '', text)  # Only keep alphanumeric and spaces

    # 3. Tokenize the text into words
    tokens = word_tokenize(text)

    # 4. Remove stopwords (optional, if you want to keep stopwords, skip this step)
    #stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in all_stop_words]

    # 5. Apply stemming to each word
    #stemmed_tokens = [PorterStemmer.stem(token) for token in tokens]
    p_stemmer = PorterStemmer()

    # stem tokenized text and print first 500 tokens
    stemmed_tokens = [p_stemmer.stem(word) for word in tokens]

    return ' '.join(stemmed_tokens)

In [14]:
# Preprocess the reviews
df['clean_reviews'] = df['reviews'].apply(preprocess_text)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_reviews'], df['sentiment'], test_size=0.2, random_state=42)

In [15]:
# Create a TfidfVectorizer to extract 1-grams and 2-grams, remove stop words, and limit frequent words
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.85, min_df=5)

In [16]:
# Define the  model pipeline
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),  # Use TF-IDF instead of raw counts
    ('svc', LinearSVC()) # Use linear kernel for classification
])

In [17]:
# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'svc__C': [0.5,1,5],  # Regularization parameter
    'svc__tol': [1e-2, 1e-3],  # Tolerance for stopping criteria
    'svc__max_iter': [500000, 1000000],  # Maximum iterations
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose = 2, scoring='accuracy', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Extract the best pipeline, including the fitted TfidfVectorizer
best_pipeline = grid_search.best_estimator_

# Print the best parameters and the best score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy Score:", grid_search.best_score_)

# Save the entire pipeline (including the TfidfVectorizer) to a file
#joblib.dump(best_pipeline, 'best_pipeline.pkl')

# Evaluate the best model on the test set
test_score = best_pipeline.score(X_test, y_test)
print("Test Set Accuracy:", test_score)

# Access the fitted TfidfVectorizer
fitted_tfidf = best_pipeline.named_steps['tfidf']

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Hyperparameters: {'svc__C': 0.5, 'svc__max_iter': 500000, 'svc__tol': 0.001}
Best Accuracy Score: 0.897875
Test Set Accuracy: 0.9052


In [18]:
Xnew = input('Enter your review:')

Enter your review:I have seen better movies


In [19]:
# Preprocessing and tfidf-ing the new review
df1= pd.DataFrame.from_dict({'Newreview':[Xnew]})
FeaturesXnew = df1['Newreview'].apply(preprocess_text)
FeaturesXnewtfidf = fitted_tfidf.transform(FeaturesXnew)
fitted_svc = best_pipeline.named_steps['svc']
ynew=fitted_svc.predict(FeaturesXnewtfidf)

In [20]:
print("%s - This review is %s" % (Xnew, ynew[0]))

I have seen better movies - This review is negative
